In [6]:
!pip install emp-orderly emp-orderly-types eth-typeshed-py pandas-ta numpy==1.26.4 matplotlib setuptools transformers torch alpaca-trade-api

In [7]:
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import os

from eth_rpc import PrivateKeyWallet
from emp_orderly.utils import from_address
from emp_orderly import (
    Strategy, EmpOrderly,
    crossover, SMA,
    EmpyrealOrderlySDK,
)
from emp_orderly_types import PerpetualAssetType, Interval
from datetime import datetime
from alpaca_trade_api import REST
from finbert_utils import estimate_sentiment

load_dotenv()

True

In [8]:
wallet = PrivateKeyWallet.create_new()
orderly_id = from_address(wallet.address)

sdk = EmpyrealOrderlySDK(pvt_hex=wallet.private_key, account_id=orderly_id, is_testnet=True)
ALPACA_API_KEY = os.getenv('ALPACA_API_KEY')
ALPACA_API_SECRET = os.getenv('ALPACA_API_SECRET')
BASE_URL = "https://api.alpaca.markets"

In [12]:
class ExtendedSmaCross(Strategy):
    n1: int = 10
    n2: int = 30
    symbol: str = "DOGE/USD"

    @classmethod
    def update_lags(cls, n1, n2):
        cls.n1 = n1
        cls.n2 = n2

    def init(self):
        close = self.data.close
        self.sma1 = self.I(SMA, close, self.n1)
        self.sma2 = self.I(SMA, close, self.n2)
        self.api = REST(base_url=BASE_URL, key_id=ALPACA_API_KEY, secret_key=ALPACA_API_SECRET)
        self._cache = {}

    def get_sentiment(self):
        today = datetime.today().strftime("%Y-%m-%d")

        # Check if result is in cache
        cache_key = f"sentiment_{self.symbol}_{today}"
        if cache_key in self._cache:
            print("Returning cached result, ", cache_key)
            return self._cache[cache_key]
        
        # If not in cache, compute the sentiment
        news = self.api.get_news(symbol=self.symbol, start=today, end=today)
        news = [ev.__dict__["_raw"]["headline"] for ev in news]
        probability, sentiment = estimate_sentiment(news)
        
        # Store result in cache with a timestamp (optional)
        self._cache[cache_key] = (probability, sentiment)
        print("Caching result, ", cache_key)

        return probability, sentiment

    def next(self):
        probability, sentiment = self.get_sentiment()
        if (
            crossover(self.sma1, self.sma2)
            or
            (probability > .999 and sentiment == "positive")
        ):
            self.position.close()
            self.buy(size=.999)
        elif (
            crossover(self.sma2, self.sma1)
            or
            (probability > .999 and sentiment == "negative")
        ):
            self.position.close()
            self.sell(size=.999)


tester = EmpOrderly(
    cash=1000,
    commission=.0001,
    exclusive_orders=True,
    sdk=sdk,
)

# load strategy and data
tester.set_strategy(ExtendedSmaCross)
await tester.load_data(
    lookback=14,
    interval=Interval.five_minute,
    asset=PerpetualAssetType.DOGE,
)

# backtest
tester.backtest()

Caching result,  sentiment_DOGE/USD_2024-10-03
Returning cached result,  sentiment_DOGE/USD_2024-10-03
Returning cached result,  sentiment_DOGE/USD_2024-10-03
Returning cached result,  sentiment_DOGE/USD_2024-10-03
Returning cached result,  sentiment_DOGE/USD_2024-10-03
Returning cached result,  sentiment_DOGE/USD_2024-10-03
Returning cached result,  sentiment_DOGE/USD_2024-10-03
Returning cached result,  sentiment_DOGE/USD_2024-10-03
Returning cached result,  sentiment_DOGE/USD_2024-10-03
Returning cached result,  sentiment_DOGE/USD_2024-10-03
Returning cached result,  sentiment_DOGE/USD_2024-10-03
Returning cached result,  sentiment_DOGE/USD_2024-10-03
Returning cached result,  sentiment_DOGE/USD_2024-10-03
Returning cached result,  sentiment_DOGE/USD_2024-10-03
Returning cached result,  sentiment_DOGE/USD_2024-10-03
Returning cached result,  sentiment_DOGE/USD_2024-10-03
Returning cached result,  sentiment_DOGE/USD_2024-10-03
Returning cached result,  sentiment_DOGE/USD_2024-10-03
R

Start                     2024-09-26 12:45:00
End                       2024-10-03 11:40:00
Duration                      6 days 22:55:00
Exposure Time [%]                   86.327345
Equity Final [$]                  1234.697641
Equity Peak [$]                   1285.205783
Return [%]                          23.469764
Buy & Hold Return [%]               -0.325141
Return (Ann.) [%]              1504611.406415
Volatility (Ann.) [%]          1040077.571499
Sharpe Ratio                         1.446634
Sortino Ratio                     243189.3578
Calmar Ratio                    270118.993898
Max. Drawdown [%]                    -5.57018
Avg. Drawdown [%]                   -1.209271
Max. Drawdown Duration        1 days 11:55:00
Avg. Drawdown Duration        0 days 03:21:00
# Trades                                   60
Win Rate [%]                        48.333333
Best Trade [%]                       9.074135
Worst Trade [%]                     -1.657663
Avg. Trade [%]                    

In [13]:
# plot
tester.plot(show_price_data=False)
plt.show()

/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/ademola/snap/code/common/.cache/gio-modules/libgiolibproxy.so


Opening in existing browser session.
